# Recommender System

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
# Initiate Spark Session

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
# Load the data

data = spark.read.csv('resources/movielens_ratings.csv', inferSchema=True, header=True)

data.show(10)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
+-------+------+------+
only showing top 10 rows



In [5]:
# Lets check the schema

data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [6]:
# Check summary statistics

data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [7]:
# We will do a training and test set... since the dataset is small

training, test = data.randomSplit([0.8, 0.2])

In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

# fit the model into training data
model = als.fit(training)

In [9]:
# Evaluate the model by computing RMSE on the test data

predictions = model.transform(test)

predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    26|-0.27544093|
|     31|   1.0|    27|  1.5712456|
|     31|   1.0|    13|  1.5783921|
|     31|   1.0|    19|  1.6556565|
|     31|   3.0|     7| 0.93598986|
|     85|   1.0|    26|  2.9671142|
|     85|   5.0|     8|   3.535202|
|     85|   4.0|     7|  1.8478123|
|     85|   1.0|     2| -5.6987123|
|     65|   1.0|    28|  3.6405218|
|     53|   1.0|     6|  3.3718083|
|     53|   1.0|    23|  0.3358389|
|     78|   1.0|    27|  1.1553127|
|     78|   1.0|     8|  1.5143912|
|     34|   1.0|    15| 0.09776801|
|     34|   1.0|     4|  0.5504533|
|     81|   1.0|    22|  0.5196725|
|     81|   1.0|    16|  1.2968026|
|     81|   1.0|    19|  1.0636601|
|     28|   1.0|     6|  2.4036698|
+-------+------+------+-----------+
only showing top 20 rows



- The way to interpret this is , for movieId 31, userId 26 gave it a rating of 1 and we predicted the rating to be 1.09

but some other ratings are way off

In [10]:
# In order to evaluate the model..

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions)

print(f'RMSE is : {rmse}')

RMSE is : 2.0061681350172518


Given the scale of movie rating is between 1 to 5, an RMSE of 2.11 is awefully bad

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [11]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

In [12]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     19|    11|
|     22|    11|
|     25|    11|
|     36|    11|
|     43|    11|
|     45|    11|
|     59|    11|
|     70|    11|
+-------+------+

